<a href="https://colab.research.google.com/github/DivyamNigam/Visualizing_Attention_embeddings/blob/main/Attention_mechanism2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [78]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
import json
from pickle import load
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from transformers import AutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch.nn.functional as F
import math
import plotly.graph_objects as go
from sklearn.decomposition import PCA

In [112]:
np.random.seed(42)
device='cuda' if torch.cuda.is_available() else 'cpu'

In [113]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



In [114]:
sentence = "I would like sit at river bank and get money from bank."

In [115]:
model = AutoModel.from_pretrained("google-bert/bert-base-uncased",output_attentions=True)

In [116]:
# Tokenize input
inputs = tokenizer(sentence, return_tensors="pt")

# Generate embeddings
outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state

# Apply PCA to each token's embedding
embeddings_3d = pca.fit_transform(last_hidden_state[0].detach().numpy())
sentence_embeddings = last_hidden_state[0, 1:-1, :]
embeddings_3d = pca.fit_transform(sentence_embeddings.detach().numpy())
print(embeddings_3d.shape)  # (19, 3)

(13, 3)


In [117]:
print(inputs)

{'input_ids': tensor([[ 101, 1045, 2052, 2066, 4133, 2012, 2314, 2924, 1998, 2131, 2769, 2013,
         2924, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [118]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.query_linear = nn.Linear(768, 768)
        self.key_linear = nn.Linear(768, 768)
        self.value_linear = nn.Linear(768, 768)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings):
        query = self.query_linear(embeddings)
        key = self.key_linear(embeddings)
        value = self.value_linear(embeddings)

        # Transpose key
        key_transposed = key.permute(0, 2, 1)

        attention_scores = torch.matmul(query, key_transposed) / math.sqrt(768)
        attention_weights = F.softmax(attention_scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        context = torch.matmul(attention_weights, value)
        return context

attention_model = Attention()

# Apply attention
contextual_embeddings_with_attention = attention_model(last_hidden_state)

# Reduce dimension using PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
embeddings_3d_with_attention = pca.fit_transform(contextual_embeddings_with_attention[0, 1:-1, :].detach().numpy())

In [128]:
# Get tokens (excluding [CLS] and [SEP])
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze())[1:-1]

# Original embeddings (excluding [CLS] and [SEP])
sentence_embeddings = last_hidden_state[0, 1:-1, :]
embeddings_3d = pca.fit_transform(sentence_embeddings.detach().numpy())

# Embeddings with attention (excluding [CLS] and [SEP])
sentence_embeddings_with_attention = self_attention_embeddings[1:-1, :]
embeddings_3d_with_attention = pca.fit_transform(sentence_embeddings_with_attention.detach().numpy())

# Original embeddings
fig = go.Figure(data=[go.Scatter3d(x=embeddings_3d[:, 0],
                                   y=embeddings_3d[:, 1],
                                   z=embeddings_3d[:, 2],
                                   mode='markers',
                                   hovertext=[f'Token {i+1}: {token}' for i, token in enumerate(tokens)])])
fig.update_layout(title='Original Embeddings', scene = dict(
                    xaxis_title='Dimension 1',
                    yaxis_title='Dimension 2',
                    zaxis_title='Dimension 3'))

# Embeddings with attention
fig_attention = go.Figure(data=[go.Scatter3d(x=embeddings_3d_with_attention[:, 0],
                                              y=embeddings_3d_with_attention[:, 1],
                                              z=embeddings_3d_with_attention[:, 2],
                                              mode='markers',
                                              hovertext=[f'Token {i+1}: {token}' for i, token in enumerate(tokens)])])
fig_attention.update_layout(title='Embeddings with Attention', scene = dict(
                    xaxis_title='Dimension 1',
                    yaxis_title='Dimension 2',
                    zaxis_title='Dimension 3'))

# Display plots
fig.show()
fig_attention.show()

In [120]:
# I would like sit at river bank and get money from bank

In [121]:
import numpy as np
import plotly.graph_objects as go
from transformers import AutoTokenizer, AutoModel
import torch

# Sentence
sentence = "I would like sit at river bank and get money from bank."

# Tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased', output_attentions=True)

# Tokenize sentence
inputs = tokenizer(sentence, return_tensors='pt')

# Get embeddings without attention
outputs = model(**inputs)
embeddings_no_attention = outputs.last_hidden_state.detach().squeeze()

# Get embeddings with attention
outputs_with_attention = model(**inputs)

# Extract attention weights from the last layer
attention_weights = outputs_with_attention.attentions[-1].detach().squeeze()

# Compute weighted sum of embeddings using attention weights
embeddings_with_attention = torch.zeros_like(embeddings_no_attention)
for i in range(embeddings_no_attention.shape[0]):
    # Average attention weights across heads
    avg_attention = torch.mean(attention_weights[:, i, :], dim=0)
    # Compute weighted sum
    embeddings_with_attention[i] = torch.sum(avg_attention.unsqueeze(-1) * embeddings_no_attention, dim=0)

# Dimensionality reduction to 3D using PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

embeddings_no_attention_3d = pca.fit_transform(embeddings_no_attention)
embeddings_with_attention_3d = pca.fit_transform(embeddings_with_attention)

# Plot embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze())

fig = go.Figure(data=[
    go.Scatter3d(
        x=embeddings_no_attention_3d[:, 0],
        y=embeddings_no_attention_3d[:, 1],
        z=embeddings_no_attention_3d[:, 2],
        mode='markers',
        text=[f"{token} ({i})" for i, token in enumerate(tokens)],
        hoverinfo='text'
    )
])

fig.update_layout(
    title='Embeddings without Attention',
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z')
)

fig.show()


fig = go.Figure(data=[
    go.Scatter3d(
        x=embeddings_with_attention_3d[:, 0],
        y=embeddings_with_attention_3d[:, 1],
        z=embeddings_with_attention_3d[:, 2],
        mode='markers',
        text=[f"{token} ({i})" for i, token in enumerate(tokens)],
        hoverinfo='text'
    )
])

fig.update_layout(
    title='Embeddings with Attention',
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z')
)

fig.show()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

